<a href="https://colab.research.google.com/github/vinay-kaushik/CS-418-DataScience/blob/master/cs_418_DataScience_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
